In [1]:
from __future__ import division
def feature_eng(y, fs, vis=False):
    
    import librosa
    import librosa.display
    import numpy as np
    import maad
    from soundsig.sound import BioSound
    import matplotlib.pyplot as plt
    import pandas as pd
    import warnings
    warnings.simplefilter('ignore')
    
    # spectrum
    ps = np.abs(np.fft.fft(y))**2
    time_step = 1/fs
    freqs = np.fft.fftfreq(y.size, time_step)
    ps = ps[0:int((len(ps)/2)-1)] # take out the negative freq
    freqs = freqs[0:int((len(freqs)/2)-1)]

    # melspectrogram
    S = librosa.feature.melspectrogram(y=y, sr=fs)
    S_dB = librosa.power_to_db(S, ref=np.max)
    
    # alpha indices
    Sxx_power,tn,fn,ext = maad.sound.spectrogram (y, fs, mode='psd')
    df_temporal_indices = maad.features.all_temporal_alpha_indices(y, fs)
    df_spectral_indices, _ = maad.features.all_spectral_alpha_indices(Sxx_power,tn,fn, extent=ext)
    df_indices = pd.concat([df_temporal_indices,df_spectral_indices], axis=1)
    
    # soundsig for modulation power spectrum
    
    myBioSound = BioSound(soundWave=y, fs=fs)
    myBioSound.mpsCalc(window=1, Norm = True)
    
    # reduce the dimension of MPS to one quadrant
    len1 = int(len(myBioSound.wf-1)/2)
    len2 = int(len(myBioSound.wt-1)/2)
    quad1 = myBioSound.mps[len1:,len2:]
    quad2 = np.fliplr(myBioSound.mps[len1:,:len2+1])
    mps = (quad1+quad2)/2
    wf = myBioSound.wf[len1:]
    wt = myBioSound.wt[len2:]
    
    if vis==True:
        plt.figure()
        plt.plot(np.arange(0,y.size/fs,1/fs), y)
        plt.show()
        
        fig, ax = plt.subplots()
        img = librosa.display.specshow(S_dB, x_axis='time',
                                 y_axis='mel', sr=fs,
                                 fmax=fs/2, ax=ax)
        fig.colorbar(img, ax=ax, format='%+2.0f dB')
        ax.set(title='Mel-frequency spectrogram')
        
        DBNOISE=100
        plt.figure()
        plt.clf()
        cmap = plt.get_cmap('jet')
        ex = (myBioSound.wt.min(), myBioSound.wt.max(), myBioSound.wf.min()*1e3, myBioSound.wf.max()*1e3)
        logMPS = 10.0*np.log10(myBioSound.mps)
        maxMPS = logMPS.max()
        minMPS = maxMPS-DBNOISE
        logMPS[logMPS < minMPS] = minMPS
        plt.imshow(logMPS, interpolation='nearest', aspect='auto', origin='lower', cmap=cmap, extent=ex)
        plt.ylabel('Spectral Frequency (Cycles/KHz)')
        plt.xlabel('Temporal Frequency (Hz)')
        plt.colorbar()
        plt.ylim((0,myBioSound.wf.max()*1e3))
        plt.title('Modulation Power Spectrum')
        plt.show()
        
    
    return ps, freqs, S_dB, df_indices, mps, wt, wf
    

In [2]:
def preproc(file, fs=16000, vis=False):
    import librosa
    import numpy as np
    import matplotlib.pyplot as plt
    import noisereduce as nr
    
    raw_y, fs = librosa.load(file, sr=fs, duration=10, mono = True)
    y_mono_rs = raw_y - np.mean(raw_y) # remove DC
    rms = np.sqrt(np.mean(y_mono_rs**2)) # get rms
    y = y_mono_rs/(rms/0.1) # normalize the rms to 0.1
    
    fg_y = nr.reduce_noise(y=y, sr=fs)
    bg_y = y - fg_y
    
    
    if vis == True: print('++++++++++++++++++++++++ raw ++++++++++++++++++++++++')
    ps, freqs, S_dB, df_indices, mps, wt, wf = feature_eng(y, fs, vis)
    if vis == True: print('++++++++++++++++++++ foreground ++++++++++++++++++++')
    ps_fg, freqs_fg, S_dB_fg, df_indices_fg, mps_fg, wt_fg, wf_fg = feature_eng(fg_y, fs, vis)
    if vis == True: print('++++++++++++++++++++ background ++++++++++++++++++++')
    ps_bg, freqs_fg, S_dB_bg, df_indices_bg, mps_bg, wt_bg, wf_bg = feature_eng(bg_y, fs, vis)
    
    output = {'fs': fs, 'y': y, 'fg_y': fg_y, 'bg_y': bg_y,
             'df_indices': df_indices, 'mps': mps, 'wt': wt, 'wf': wf, 
             'df_indices_fg': df_indices_fg, 'mps_fg': mps_fg,
             'df_indices_bg': df_indices_bg, 'mps_bg': mps_bg}
    
    return output
    

In [3]:
# scan data directories

import glob
import pandas as pd
nature_file_list = []
nature_file_list += glob.glob('../data/raw/GoogleAudioSet_unbalanced_list/Outside, rural or natural/*')
nature_file_list += glob.glob('../data/raw/GoogleAudioSet_eval_list/Outside, rural or natural/*')

city_file_list = []
city_file_list += glob.glob('../data/raw/GoogleAudioSet_unbalanced_list/Outside, urban or manmade/*')
city_file_list += glob.glob('../data/raw/GoogleAudioSet_eval_list/Outside, urban or manmade/*')


remove_list_nature = glob.glob('../data/raw/GoogleAudioSet_unbalanced_list/Outside, rural or natural/AE0v7LesLZo*')
remove_list_city = glob.glob('../data/raw/GoogleAudioSet_unbalanced_list/Outside, urban or manmade/OPyovt30GPQ*')
remove_list_city += glob.glob('../data/raw/GoogleAudioSet_unbalanced_list/Outside, urban or manmade/CBmGYSOoeto*')


nature_file_list = [ele for ele in nature_file_list if ele not in remove_list_nature]
city_file_list = [ele for ele in city_file_list if ele not in remove_list_city]


In [4]:
# # scan data directories
# import glob
# import pandas as pd

# nature_file_list = []
# nature_file_list += glob.glob('../data/raw/AmbisonicSoundLibrary/nature/*')
# nature_file_list += glob.glob('../data/raw/GoogleAudioSet/Outside, rural or natural/*')
# nature_file_list += glob.glob('../data/raw/youtube/NatureSoundscapes/*.m4a')
# nature_file_list += glob.glob('../data/raw/youtube/NomadicAmbience_nature/*')
# nature_file_list += glob.glob('../data/raw/S2L_LULC/non_urban/*')
# nature_file_list += glob.glob('../data/raw/S2L_LULC/urban_0_25/*')

# remove_list_nature = glob.glob('../data/raw/youtube/NatureSoundscapes/xRrqcK46roE_*.m4a')
# nature_file_list = [ele for ele in nature_file_list if ele not in remove_list_nature]


# city_file_list = []
# city_file_list += glob.glob('../data/raw/GoogleAudioSet/Outside, urban or manmade/*')
# city_file_list += glob.glob('../data/raw/youtube/NomadicAmbience_city/*')
# city_file_list += glob.glob('../data/raw/S2L_LULC/urban_26_100/*')

# remove_list_city += glob.glob('../data/raw/youtube/NomadicAmbience_city/F5t-G50wnCo_*.m4a')
# city_file_list = [ele for ele in city_file_list if ele not in remove_list_city]


In [5]:
# # as there are too many files in SONYC, so only a few are sampled
# from random import seed, sample
# len_diff = len(nature_file_list)-len(city_file_list) # how many SONYC files need to add to make the dataset balanced

# seed(23)
# SONYC_file_list = sample(glob.glob('../data/raw/SONYC/**/*.wav', recursive=True),len_diff)
# city_file_list += SONYC_file_list

# pd.DataFrame({'nature_file_list': nature_file_list}).to_csv('../data/raw/nature_file_list.csv')
# pd.DataFrame({'city_file_list': city_file_list}).to_csv('../data/raw/city_file_list.csv')

In [6]:
# print('There are '+str(len(nature_file_list))+' files per category.')

In [7]:
# testing cell
# preproc(city_file_list[0])

In [8]:
import time
import pickle
import os

from joblib import Parallel, delayed

def run_preproc(file_name):
    save_file_name = '../data/interim/'+file_name[12:-4]+'.pkl'
    if not os.path.isfile(save_file_name): # run the script only if the file does not exist
        if not os.path.exists(save_file_name.rsplit('/', 1)[0]): # create the folder if the folder does not exist
            os.makedirs(save_file_name.rsplit('/', 1)[0]) # extract the folder of the file path
            

        start_time = time.time()

        output = preproc(file_name, vis = False)
        f = open(save_file_name,'wb') # create a binary pickle file 
        pickle.dump(output,f)
        f.close()

        print("--- %s seconds ---" % (time.time() - start_time))
        

# run the process in parallel
Parallel(n_jobs=-1)(delayed(run_preproc)(file_name) for file_name in nature_file_list+city_file_list)

--- 14.84307312965393 seconds ---


/Users/andrewchang/opt/anaconda3/envs/AcousticEnv/lib/python3.8/site-packages/joblib/externals/loky/process_executor.py:700: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


--- 14.951782941818237 seconds ---
--- 14.926563024520874 seconds ---
--- 14.924899816513062 seconds ---
--- 14.95044207572937 seconds ---
--- 14.994908094406128 seconds ---
--- 15.099287748336792 seconds ---
--- 14.839067697525024 seconds ---
--- 6.941596031188965 seconds ---
--- 13.052984952926636 seconds ---
--- 13.204126119613647 seconds ---
--- 13.65344786643982 seconds ---
--- 13.823923826217651 seconds ---
--- 13.884681940078735 seconds ---
--- 13.648509979248047 seconds ---
--- 13.92968487739563 seconds ---
--- 13.937122106552124 seconds ---
--- 13.708305358886719 seconds ---
--- 13.48140001296997 seconds ---
--- 13.591721057891846 seconds ---
--- 13.514524936676025 seconds ---
--- 8.437086820602417 seconds ---
--- 8.526113986968994 seconds ---
--- 17.577263832092285 seconds ---
--- 18.735777139663696 seconds ---
--- 20.37351894378662 seconds ---
--- 18.25896906852722 seconds ---
--- 17.721564054489136 seconds ---
--- 16.08249807357788 seconds ---


[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,

In [9]:
# for file_name in nature_file_list+city_file_list:
#     print(file_name)
#     run_preproc(file_name)

--- 14.812633991241455 seconds ---
--- 8.872724056243896 seconds ---
--- 8.688029289245605 seconds ---
--- 8.522797107696533 seconds ---
--- 8.61698293685913 seconds ---
--- 8.717271089553833 seconds ---
--- 8.567198038101196 seconds ---
--- 8.437108993530273 seconds ---
--- 9.000672101974487 seconds ---
--- 9.673925161361694 seconds ---
--- 8.621731758117676 seconds ---
--- 8.779838800430298 seconds ---
--- 9.40567922592163 seconds ---
--- 9.29550838470459 seconds ---
--- 8.689182758331299 seconds ---
--- 8.849771976470947 seconds ---
--- 9.400900840759277 seconds ---
--- 8.697858810424805 seconds ---
--- 8.743648052215576 seconds ---
--- 9.841366052627563 seconds ---
--- 8.654734134674072 seconds ---
--- 8.677349090576172 seconds ---
--- 9.267712831497192 seconds ---
--- 10.047835111618042 seconds ---
--- 10.379003286361694 seconds ---
--- 10.230626106262207 seconds ---
--- 9.206184148788452 seconds ---
--- 8.628946781158447 seconds ---
--- 8.914677619934082 seconds ---
--- 9.1641659

--- 18.015385150909424 seconds ---
--- 12.571498155593872 seconds ---
--- 10.104597568511963 seconds ---
--- 10.391664028167725 seconds ---
--- 8.40487003326416 seconds ---
--- 9.263502836227417 seconds ---
--- 8.686405181884766 seconds ---
--- 8.326955080032349 seconds ---
--- 9.140437602996826 seconds ---
--- 8.806299924850464 seconds ---
--- 8.70104694366455 seconds ---
--- 9.064502954483032 seconds ---
--- 9.810397148132324 seconds ---
--- 8.873189926147461 seconds ---
--- 8.791582107543945 seconds ---
--- 9.166736841201782 seconds ---
--- 9.380753993988037 seconds ---
--- 9.155711889266968 seconds ---
--- 8.592296838760376 seconds ---
--- 9.59620976448059 seconds ---
--- 8.568602085113525 seconds ---
--- 8.341557025909424 seconds ---
--- 9.724366903305054 seconds ---
--- 9.055389165878296 seconds ---
--- 8.916177988052368 seconds ---
--- 9.142542123794556 seconds ---
--- 9.763267040252686 seconds ---
--- 10.923285961151123 seconds ---
--- 9.704362869262695 seconds ---
--- 9.916828